In [1]:
!pip install tf-nightly

     |████████████████████████████████| 324.8 MB 3.0 kB/s 
     |████████████████████████████████| 6.7 MB 37.1 MB/s 
     |████████████████████████████████| 459 kB 40.6 MB/s 


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/part1dataset/train.tfrecords
/kaggle/input/validationtfrecords/val.tfrecords


In [3]:
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import glob
import cv2
import sys
from random import shuffle

In [4]:
 class DataSequenceReader():
    def __init__(self, batch_size, num_epochs):
        self.batch_size = batch_size
        self.num_epochs = num_epochs
    
    def parse_sequence(self, sequence_example):

        sequence_features = {'Images': tf.io.FixedLenSequenceFeature([], dtype=tf.string),
                          'Labels': tf.io.FixedLenSequenceFeature([], dtype=tf.int64)}

        context_features = {'length': tf.io.FixedLenFeature([], dtype=tf.int64),
                         'height': tf.io.FixedLenFeature([], dtype=tf.int64),
                         'width': tf.io.FixedLenFeature([], dtype=tf.int64),
                         'depth': tf.io.FixedLenFeature([], dtype=tf.int64)}
        
        context, sequence = tf.io.parse_single_sequence_example(
            sequence_example, context_features=context_features, sequence_features=sequence_features)
        # get features context
        seq_length = tf.cast(context['length'], dtype = tf.int32)
        im_height = tf.cast(context['height'], dtype = tf.int32)
        im_width = tf.cast(context['width'], dtype = tf.int32)
        im_depth = tf.cast(context['depth'], dtype = tf.int32)

        # encode image
        image = tf.map_fn(lambda x: tf.io.decode_jpeg(x, 1), sequence['Images'], tf.uint8)
        image = tf.cast(image, dtype = tf.float32)
        image = tf.reshape(image, shape=(seq_length, im_height, im_width, im_depth))
        image = tf.image.per_image_standardization(image)
        label = tf.cast(sequence['Labels'], dtype = tf.int32)

        return image, label[0]
    
    def read_batch(self, filename):
        

        dataset = tf.data.TFRecordDataset(filename)
        dataset = dataset.map(self.parse_sequence)

        dataset = dataset.shuffle(buffer_size=10000)

        dataset = dataset.batch(self.batch_size, drop_remainder=True)
        
#         dataset = dataset.repeat(self.num_epochs)

        return dataset

In [5]:
# My model
# Convolutions
conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='elu', name="conv1", data_format='channels_last')
pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='elu', name="conv2", data_format='channels_last')
pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')

conv3 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='elu', name="conv3", data_format='channels_last')
pool3 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')

# LSTM & Flatten
convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
flatten =  tf.keras.layers.Flatten(name="flatten")

# Dense layers
d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
out = tf.keras.layers.Dense(27, activation='softmax', name="output")

#BatchNorm
b1 = tf.keras.layers.BatchNormalization(axis=-1)
b2 = tf.keras.layers.BatchNormalization(axis=-1)
b3 = tf.keras.layers.BatchNormalization(axis=-1)



#Dropout
drop1 = tf.keras.layers.Dropout(.5)
drop2 = tf.keras.layers.Dropout(.5)
drop3 = tf.keras.layers.Dropout(.5)



model = Sequential()
model.add(conv1)
model.add(b1)
model.add(drop1)
model.add(pool1)
model.add(conv2)
model.add(b2)
model.add(drop2)
model.add(pool2)
model.add(conv3)
model.add(b3)
model.add(drop3)
model.add(pool3)
model.add(convLSTM)
model.add(flatten)
model.add(d1)
model.add(out)

In [6]:
# with tf.device('/device:GPU:0'):
# #with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
#   model = Conv3DModel()
# Loss
train_loss = tf.keras.metrics.Mean(name='train_loss')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
# Accuracy
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [8]:
@tf.function
def train_step(image, targets):
    with tf.GradientTape() as tape:
        # Make a prediction on all the batch
        predictions = model(image)
        # Get the error/loss on these predictions
        loss = loss_fn(targets, predictions)
    # Compute the gradient which respect to the loss
    grads = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)
    train_accuracy(targets, predictions)

In [9]:
@tf.function
def valid_step(image, targets):
    predictions = model(image)
    t_loss = loss_fn(targets, predictions)
    # Set the metrics for the test
    valid_loss(t_loss)
    valid_accuracy(targets, predictions)

In [10]:
batch_size =32
num_epochs = 20
data = DataSequenceReader(batch_size, num_epochs)

train_batch = data.read_batch('/kaggle/input/part1dataset/train.tfrecords')
val_batch = data.read_batch('/kaggle/input/validationtfrecords/val.tfrecords')

count=0

training_acc =[]
validation_acc = []


with tf.device('/device:GPU:0'):

    for i in range(num_epochs):
        for images, labels in train_batch:
            train_step(images, labels)


        for images, labels in val_batch:
            valid_step(images, labels)

        print("--Train step done--")
        count=count+1
        print(count)

        print(train_loss.result(), train_accuracy.result()*100)
        print(valid_loss.result(), valid_accuracy.result()*100)

        training_acc.append(float(train_accuracy.result()*100))
        validation_acc.append(float(valid_accuracy.result()*100))


        train_loss.reset_states()
        train_accuracy.reset_states()

        valid_loss.reset_states()
        valid_accuracy.reset_states()

--Train step done--
1
tf.Tensor(1.8736979, shape=(), dtype=float32) tf.Tensor(41.5463, shape=(), dtype=float32)
tf.Tensor(1.3105254, shape=(), dtype=float32) tf.Tensor(57.01291, shape=(), dtype=float32)
--Train step done--
2
tf.Tensor(1.1811628, shape=(), dtype=float32) tf.Tensor(60.883503, shape=(), dtype=float32)
tf.Tensor(1.056187, shape=(), dtype=float32) tf.Tensor(65.83774, shape=(), dtype=float32)
--Train step done--
3
tf.Tensor(0.9769924, shape=(), dtype=float32) tf.Tensor(67.562096, shape=(), dtype=float32)
tf.Tensor(0.97520566, shape=(), dtype=float32) tf.Tensor(67.928406, shape=(), dtype=float32)
--Train step done--
4
tf.Tensor(0.8397965, shape=(), dtype=float32) tf.Tensor(71.98805, shape=(), dtype=float32)
tf.Tensor(0.93669873, shape=(), dtype=float32) tf.Tensor(69.703636, shape=(), dtype=float32)
--Train step done--
5
tf.Tensor(0.73388296, shape=(), dtype=float32) tf.Tensor(75.31892, shape=(), dtype=float32)
tf.Tensor(0.9155842, shape=(), dtype=float32) tf.Tensor(71.12676, 

In [11]:
model.save('/kaggle/working/my_model', save_format='h5')


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (32, 28, 62, 62, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (32, 28, 62, 62, 32)      128       
_________________________________________________________________
dropout (Dropout)            (32, 28, 62, 62, 32)      0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (32, 14, 31, 31, 32)      0         
_________________________________________________________________
conv2 (Conv3D)               (32, 12, 29, 29, 64)      55360     
_________________________________________________________________
batch_normalization_1 (Batch (32, 12, 29, 29, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (32, 12, 29, 29, 64)      0